# Importing Libraries

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Importing Data Sets

In [4]:
path = r'C:\Users\Aricia\MM-2020 Instacart Basket Analysis'

In [5]:
# Importing updated Orders Products Merge 
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_ords_prods_merge.pkl'))

In [6]:
# Checking number of columns and rows
ords_prods_merge.shape

(32434212, 19)

In [7]:
# Checking columns name
ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', '_merge', 'Merge_flag', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day'],
      dtype='object')

## 2 Aggregating Data with Agg

In [8]:
# Split data into groups based on department and apply add to obtain mean values for the order number
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


# 3 Analysing Results:
All the means shown have different results when comparing to the total dataframe against the subset. Some differences are more than others, but they are relatively similar. 

# 4 Create Loyalty Flag

In [9]:
# Aggregating data with transform
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\446144649.py:2: FutureWarning: The provided callable <function max at 0x000001D7C47047C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [10]:
ords_prods_merge.head(5)

C:\Users\Aricia\anaconda3\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,Merge_flag,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Regularly busy,Regular days,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Regularly busy,Regular days,Average orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Busiest day,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,-1.0,10,0,both,both,Mid-range product,Regularly busy,Slowest days,Average orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Least busy,Slowest days,Average orders,3


In [11]:
# Creating flag for loyal customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [12]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [13]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
# Checking frequency of loyalty-flags
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

## 5 Understanding spend habits per group

In [15]:
# Group by loyalty flag and prices
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10293737.0,10.388747,327.864108,1.0,4.2,7.4,11.2,99999.0
New customer,6249398.0,13.294370,597.301692,1.0,4.2,7.4,11.3,99999.0
Regular customer,15891077.0,12.496203,539.478009,1.0,4.2,7.4,11.3,99999.0


When comparing each loyalty category by product prices, we can notice that although loyal customers buy frequently, their average price product is lower. On the other hand, new customers tend to spend more on products. It is important to notice though that loyal customer deviation is lower than the others, so their data is more consistent. Their minimum and max values are all the same, which is a really big difference and would require additional analysis as it can play a significant part on mean and std.

## 6 Spender_type flag

In [16]:
# Aggregating data with transform - user_id and price
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\4020609015.py:2: FutureWarning: The provided callable <function mean at 0x000001D7C47051C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
ords_prods_merge.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,reordered,_merge,Merge_flag,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,0,both,both,Mid-range product,Regularly busy,Regular days,Most orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,both,both,Mid-range product,Regularly busy,Regular days,Average orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,0,both,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,0,both,both,Mid-range product,Regularly busy,Slowest days,Average orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,1,both,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414


In [18]:
# Creating flag for spender type csts
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spender_type'] = 'Low spender'

In [19]:
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spender_type'] = 'High spender'

In [20]:
# Checking frequency of spender types
ords_prods_merge['spender_type'].value_counts(dropna = False)

spender_type
Low spender     31798751
High spender      635461
Name: count, dtype: int64

## 7 Creating an order_frequency flag

In [21]:
# Aggregating data with transform - user_id and price
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\3702705143.py:2: FutureWarning: The provided callable <function median at 0x000001D7C4867060> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [22]:
ords_prods_merge.head(5)

C:\Users\Aricia\anaconda3\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
C:\Users\Aricia\anaconda3\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,Merge_flag,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spender_type,order_frequency
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,Regular days,Most orders,32,Regular customer,6.935811,Low spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,Regular days,Average orders,32,Regular customer,6.935811,Low spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,6.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Regularly busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0


In [23]:
# Creating flag for order frequency csts
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [24]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [25]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [26]:
# Checking frequency of orders per cst
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        22815325
Regular customer          6928900
Non-frequent customer     2689987
Name: count, dtype: int64

In [31]:
# Optimization code for memory issues to be run before saving the pickle file 
def optimize_memory(df):
    for col in df.columns:
        col_type = df[col].dtype
        
        # -------------------------------------------------------------
        # 1. SKIP Check: Skip if object, categorical, or has missing values (NaN)
        # -------------------------------------------------------------
        # If it's a generic object (string) or a category, skip the numerical min/max check
        if pd.api.types.is_object_dtype(col_type) or pd.api.types.is_categorical_dtype(col_type):
            
            # Special handling for strings/objects to convert to category
            # This is your existing memory-saving logic for strings
            if pd.api.types.is_object_dtype(col_type):
                # Check for low cardinality (adjust threshold if needed)
                if len(df[col].unique()) / len(df[col]) < 0.5: 
                    df[col] = df[col].astype('category')
            
            # Move to the next column
            continue
        
        # -------------------------------------------------------------
        # 2. NUMERICAL Downcasting Logic (Only runs for int/float types)
        # -------------------------------------------------------------
        
        # Check for NaN/missing values before running min/max on numerical types
        if df[col].isnull().any():
            print(f"Skipping numerical downcasting for column '{col}' due to NaN values.")
            # Skip to the next column
            continue
            
        c_min = df[col].min()
        c_max = df[col].max()

        if str(col_type)[:3] == 'int':
            # Downcast integer types
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            # ... and so on for int32 (you can add this check if desired)
            
        elif str(col_type)[:5] == 'float':
            # Downcast float types
            df[col] = df[col].astype(np.float32)
            
    return df

In [32]:
# Execute function
df_ords_prods_optimized = optimize_memory(ords_prods_merge)

C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\3124988887.py:10: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(col_type) or pd.api.types.is_categorical_dtype(col_type):
C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\3124988887.py:10: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(col_type) or pd.api.types.is_categorical_dtype(col_type):
C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\3124988887.py:10: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(col_type) or pd.api.types.is_categorical_dtype(col_type):
C:\Users\Aricia\AppData\Local\Temp\ipykernel_13524\3124988887.py:10: Depreca

In [33]:
# Exporting the data to pkl
df_ords_prods_optimized.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_optimized.pkl'))